# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))
print('Max number of words in a sentence: {}'.format(np.max(word_counts)))
print('Min number of words in a sentence: {}'.format(np.min(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028
Max number of words in a sentence: 17
Min number of words in a sentence: 0

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .


## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of each sentence from `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    EOS = target_vocab_to_int['<EOS>']
    source_sentences = [sentence.split() for sentence in source_text.split('\n')]
    source_id_text = [[source_vocab_to_int[word] for word in sentence] for sentence in source_sentences]
    target_sentences = [sentence.split() for sentence in target_text.split('\n')]
    target_id_text = [[target_vocab_to_int[word] for word in sentence] + [EOS] for sentence in target_sentences]
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) in [LooseVersion('1.0.0'), LooseVersion('1.0.1')], 'This project requires TensorFlow version 1.0  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1


/Users/safdar/anaconda3/envs/dl-python3.5/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoding_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.

Return the placeholders in the following the tuple (Input, Targets, Learing Rate, Keep Probability)

In [8]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate, keep probability)
    """
    # TODO: Implement Function
    inputs_ = tf.placeholder(tf.int32, [None, None], name="input")
    targets_ = tf.placeholder(tf.int32, [None, None], name="target")
    learning_rate_ = tf.placeholder(tf.float32, name="learning_rate")
    keep_prob_ = tf.placeholder(tf.float32, name="keep_prob")
    return inputs_, targets_, learning_rate_, keep_prob_

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoding Input
Implement `process_decoding_input` using TensorFlow to remove the last word id from each batch in `target_data` and concat the GO ID to the beginning of each batch.

In [9]:
# Here we are preparing the training input to the decoder. A '<GO>' character will mark the beginning of the sentence
# and we only need to provide len(sentence)-1 words, since the last word will never be provided as input to the decoder.
def process_decoding_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for decoding
    :param target_data: Target Placeholder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    slice_ = tf.strided_slice(target_data, begin=[0,0], end=[batch_size, -1], strides=[1,1])
    processed_target_data = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), slice_], 1)
#     print (target_data.get_shape())
#     print (processed_target_data.get_shape())
#     print (batch_size)
    
    return processed_target_data

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_decoding_input(process_decoding_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer using [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn).

In [10]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :return: RNN state
    """
    # TODO: Implement Function
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    layered_rnn = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)
    dropout = tf.contrib.rnn.DropoutWrapper(layered_rnn, output_keep_prob=keep_prob)
    _, encoder_state = tf.nn.dynamic_rnn(layered_rnn, rnn_inputs, dtype=tf.float32)
    
    return encoder_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create training logits using [`tf.contrib.seq2seq.simple_decoder_fn_train()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_train) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder).  Apply the `output_fn` to the [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) outputs.

In [11]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param sequence_length: Sequence Length
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Train Logits
    """
    # TODO: Implement Function
    train_decoder_fn = tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
    train_decoder_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                                train_decoder_fn, 
                                                                dec_embed_input,
                                                                sequence_length,
                                                                scope=decoding_scope)
    train_logits = output_fn(train_decoder_pred)
        
    return train_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference logits using [`tf.contrib.seq2seq.simple_decoder_fn_inference()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_inference) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder). 

In [12]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param maximum_length: The maximum allowed time steps to decode
    :param vocab_size: Size of vocabulary
    :param decoding_scope: TensorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Inference Logits
    """
    # TODO: Implement Function
#     decoding_scope.reuse_variables()
    infer_decoder_fn = tf.contrib.seq2seq.simple_decoder_fn_inference(output_fn,
                                                                     encoder_state,
                                                                     dec_embeddings,
                                                                     start_of_sequence_id,
                                                                     end_of_sequence_id,
                                                                     maximum_length,
                                                                     vocab_size)
    inference_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell,
                                                                    infer_decoder_fn,
                                                                    scope=decoding_scope)
    return inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

- Create RNN cell for decoding using `rnn_size` and `num_layers`.
- Create the output fuction using [`lambda`](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to transform it's input, logits, to class logits.
- Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)` function to get the training logits.
- Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, decoding_scope, output_fn, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [13]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, target_vocab_to_int, keep_prob):
    """
    Create decoding layer
    :param dec_embed_input: Decoder embedded input
    :param dec_embeddings: Decoder embeddings
    :param encoder_state: The encoded state
    :param vocab_size: Size of vocabulary
    :param sequence_length: Sequence Length
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training Logits, Inference Logits)
    """
    # TODO: Implement Function
    start_of_sequence_id = target_vocab_to_int['<GO>']
    end_of_sequence_id = target_vocab_to_int['<EOS>']
    
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    layered_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell] * num_layers)
    dec_cell = tf.contrib.rnn.DropoutWrapper(layered_cell, output_keep_prob=keep_prob)
    
    with tf.variable_scope("decoding") as decoding_scope:
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, vocab_size, None, scope=decoding_scope)
        train_logits = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length,
                                           decoding_scope, output_fn, keep_prob)
        decoding_scope.reuse_variables()
        inference_logits = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                                                end_of_sequence_id, sequence_length-1, vocab_size, decoding_scope,
                                                output_fn, keep_prob)
    
    return train_logits, inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob)`.
- Process target data using your `process_decoding_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)`.

In [14]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param sequence_length: Sequence Length
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training Logits, Inference Logits)
    """
    # TODO: Implement Function
    encoder_embedding_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, enc_embedding_size)
    encoder_state = encoding_layer(encoder_embedding_input, rnn_size, num_layers, keep_prob)
    
    processed_target_data = process_decoding_input(target_data, target_vocab_to_int, batch_size)
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, dec_embedding_size]))
    dec_embedding_input = tf.nn.embedding_lookup(dec_embeddings, processed_target_data)
    
    train_logits, inference_logits = decoding_layer(dec_embedding_input, 
                                                    dec_embeddings, 
                                                    encoder_state, 
                                                    target_vocab_size,
                                                    sequence_length,
                                                    rnn_size,
                                                    num_layers, 
                                                    target_vocab_to_int, 
                                                    keep_prob)
    
    return train_logits, inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability

In [25]:
# Number of Epochs
epochs = 3
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 1.0

### Build the Graph
Build the graph using the neural network you implemented.

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_source_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob = model_inputs()
    sequence_length = tf.placeholder_with_default(max_source_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)
    
    train_logits, inference_logits = seq2seq_model(
        tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(source_vocab_to_int), len(target_vocab_to_int),
        encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, target_vocab_to_int)

    tf.identity(inference_logits, 'logits')
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            targets,
            tf.ones([input_shape[0], sequence_length]))

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forums to see if anyone is having the same problem.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import time

def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1]), (0,0)],
            'constant')

    return np.mean(np.equal(target, np.argmax(logits, 2)))

train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]

valid_source = helper.pad_sentence_batch(source_int_text[:batch_size])
valid_target = helper.pad_sentence_batch(target_int_text[:batch_size])

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch) in enumerate(
                helper.batch_data(train_source, train_target, batch_size)):
            start_time = time.time()
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 sequence_length: target_batch.shape[1],
                 keep_prob: keep_probability})
            
            batch_train_logits = sess.run(
                inference_logits,
                {input_data: source_batch, keep_prob: 1.0})
            batch_valid_logits = sess.run(
                inference_logits,
                {input_data: valid_source, keep_prob: 1.0})
                
            train_acc = get_accuracy(target_batch, batch_train_logits)
            valid_acc = get_accuracy(np.array(valid_target), batch_valid_logits)
            end_time = time.time()
            print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'
                  .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    0/1077 - Train Accuracy:  0.294, Validation Accuracy:  0.305, Loss:  5.883
Epoch   0 Batch    1/1077 - Train Accuracy:  0.211, Validation Accuracy:  0.277, Loss:  4.596
Epoch   0 Batch    2/1077 - Train Accuracy:  0.171, Validation Accuracy:  0.277, Loss:  4.445
Epoch   0 Batch    3/1077 - Train Accuracy:  0.270, Validation Accuracy:  0.335, Loss:  3.917
Epoch   0 Batch    4/1077 - Train Accuracy:  0.263, Validation Accuracy:  0.337, Loss:  3.721
Epoch   0 Batch    5/1077 - Train Accuracy:  0.300, Validation Accuracy:  0.342, Loss:  3.634
Epoch   0 Batch    6/1077 - Train Accuracy:  0.271, Validation Accuracy:  0.327, Loss:  3.472
Epoch   0 Batch    7/1077 - Train Accuracy:  0.265, Validation Accuracy:  0.341, Loss:  3.432
Epoch   0 Batch    8/1077 - Train Accuracy:  0.269, Validation Accuracy:  0.339, Loss:  3.388
Epoch   0 Batch    9/1077 - Train Accuracy:  0.278, Validation Accuracy:  0.340, Loss:  3.284
Epoch   0 Batch   10/1077 - Train Accuracy:  0.249, Validati

Epoch   0 Batch   88/1077 - Train Accuracy:  0.503, Validation Accuracy:  0.529, Loss:  1.232
Epoch   0 Batch   89/1077 - Train Accuracy:  0.519, Validation Accuracy:  0.550, Loss:  1.240
Epoch   0 Batch   90/1077 - Train Accuracy:  0.505, Validation Accuracy:  0.562, Loss:  1.231
Epoch   0 Batch   91/1077 - Train Accuracy:  0.565, Validation Accuracy:  0.553, Loss:  1.052
Epoch   0 Batch   92/1077 - Train Accuracy:  0.555, Validation Accuracy:  0.558, Loss:  1.143
Epoch   0 Batch   93/1077 - Train Accuracy:  0.533, Validation Accuracy:  0.558, Loss:  1.130
Epoch   0 Batch   94/1077 - Train Accuracy:  0.517, Validation Accuracy:  0.545, Loss:  1.046
Epoch   0 Batch   95/1077 - Train Accuracy:  0.525, Validation Accuracy:  0.544, Loss:  1.112
Epoch   0 Batch   96/1077 - Train Accuracy:  0.526, Validation Accuracy:  0.573, Loss:  1.049
Epoch   0 Batch   97/1077 - Train Accuracy:  0.529, Validation Accuracy:  0.581, Loss:  1.060
Epoch   0 Batch   98/1077 - Train Accuracy:  0.584, Validati

Epoch   0 Batch  176/1077 - Train Accuracy:  0.587, Validation Accuracy:  0.621, Loss:  0.643
Epoch   0 Batch  177/1077 - Train Accuracy:  0.631, Validation Accuracy:  0.609, Loss:  0.641
Epoch   0 Batch  178/1077 - Train Accuracy:  0.614, Validation Accuracy:  0.609, Loss:  0.613
Epoch   0 Batch  179/1077 - Train Accuracy:  0.629, Validation Accuracy:  0.608, Loss:  0.646
Epoch   0 Batch  180/1077 - Train Accuracy:  0.614, Validation Accuracy:  0.608, Loss:  0.631
Epoch   0 Batch  181/1077 - Train Accuracy:  0.619, Validation Accuracy:  0.618, Loss:  0.645
Epoch   0 Batch  182/1077 - Train Accuracy:  0.631, Validation Accuracy:  0.625, Loss:  0.599
Epoch   0 Batch  183/1077 - Train Accuracy:  0.633, Validation Accuracy:  0.626, Loss:  0.625
Epoch   0 Batch  184/1077 - Train Accuracy:  0.651, Validation Accuracy:  0.629, Loss:  0.596
Epoch   0 Batch  185/1077 - Train Accuracy:  0.593, Validation Accuracy:  0.629, Loss:  0.629
Epoch   0 Batch  186/1077 - Train Accuracy:  0.602, Validati

Epoch   0 Batch  264/1077 - Train Accuracy:  0.690, Validation Accuracy:  0.665, Loss:  0.442
Epoch   0 Batch  265/1077 - Train Accuracy:  0.666, Validation Accuracy:  0.664, Loss:  0.435
Epoch   0 Batch  266/1077 - Train Accuracy:  0.689, Validation Accuracy:  0.669, Loss:  0.434
Epoch   0 Batch  267/1077 - Train Accuracy:  0.702, Validation Accuracy:  0.665, Loss:  0.407
Epoch   0 Batch  268/1077 - Train Accuracy:  0.703, Validation Accuracy:  0.664, Loss:  0.428
Epoch   0 Batch  269/1077 - Train Accuracy:  0.653, Validation Accuracy:  0.658, Loss:  0.457
Epoch   0 Batch  270/1077 - Train Accuracy:  0.695, Validation Accuracy:  0.661, Loss:  0.438
Epoch   0 Batch  271/1077 - Train Accuracy:  0.720, Validation Accuracy:  0.656, Loss:  0.405
Epoch   0 Batch  272/1077 - Train Accuracy:  0.700, Validation Accuracy:  0.661, Loss:  0.450
Epoch   0 Batch  273/1077 - Train Accuracy:  0.672, Validation Accuracy:  0.674, Loss:  0.413
Epoch   0 Batch  274/1077 - Train Accuracy:  0.745, Validati

Epoch   0 Batch  352/1077 - Train Accuracy:  0.753, Validation Accuracy:  0.752, Loss:  0.302
Epoch   0 Batch  353/1077 - Train Accuracy:  0.769, Validation Accuracy:  0.740, Loss:  0.318
Epoch   0 Batch  354/1077 - Train Accuracy:  0.749, Validation Accuracy:  0.722, Loss:  0.321
Epoch   0 Batch  355/1077 - Train Accuracy:  0.754, Validation Accuracy:  0.723, Loss:  0.297
Epoch   0 Batch  356/1077 - Train Accuracy:  0.750, Validation Accuracy:  0.726, Loss:  0.285
Epoch   0 Batch  357/1077 - Train Accuracy:  0.741, Validation Accuracy:  0.715, Loss:  0.275
Epoch   0 Batch  358/1077 - Train Accuracy:  0.759, Validation Accuracy:  0.740, Loss:  0.294
Epoch   0 Batch  359/1077 - Train Accuracy:  0.812, Validation Accuracy:  0.754, Loss:  0.279
Epoch   0 Batch  360/1077 - Train Accuracy:  0.769, Validation Accuracy:  0.743, Loss:  0.272
Epoch   0 Batch  361/1077 - Train Accuracy:  0.811, Validation Accuracy:  0.744, Loss:  0.306
Epoch   0 Batch  362/1077 - Train Accuracy:  0.759, Validati

...

Epoch   2 Batch  664/1077 - Train Accuracy:  0.953, Validation Accuracy:  0.907, Loss:  0.022
Epoch   2 Batch  665/1077 - Train Accuracy:  0.962, Validation Accuracy:  0.911, Loss:  0.018
Epoch   2 Batch  666/1077 - Train Accuracy:  0.953, Validation Accuracy:  0.914, Loss:  0.035
Epoch   2 Batch  667/1077 - Train Accuracy:  0.965, Validation Accuracy:  0.924, Loss:  0.029
Epoch   2 Batch  668/1077 - Train Accuracy:  0.959, Validation Accuracy:  0.925, Loss:  0.023
Epoch   2 Batch  669/1077 - Train Accuracy:  0.953, Validation Accuracy:  0.932, Loss:  0.022
Epoch   2 Batch  670/1077 - Train Accuracy:  0.961, Validation Accuracy:  0.926, Loss:  0.023
Epoch   2 Batch  671/1077 - Train Accuracy:  0.949, Validation Accuracy:  0.931, Loss:  0.027
Epoch   2 Batch  672/1077 - Train Accuracy:  0.969, Validation Accuracy:  0.926, Loss:  0.025
Epoch   2 Batch  673/1077 - Train Accuracy:  0.965, Validation Accuracy:  0.922, Loss:  0.026
Epoch   2 Batch  674/1077 - Train Accuracy:  0.938, Validati

Epoch   2 Batch  752/1077 - Train Accuracy:  0.953, Validation Accuracy:  0.940, Loss:  0.020
Epoch   2 Batch  753/1077 - Train Accuracy:  0.950, Validation Accuracy:  0.936, Loss:  0.026
Epoch   2 Batch  754/1077 - Train Accuracy:  0.943, Validation Accuracy:  0.938, Loss:  0.026
Epoch   2 Batch  755/1077 - Train Accuracy:  0.948, Validation Accuracy:  0.938, Loss:  0.033
Epoch   2 Batch  756/1077 - Train Accuracy:  0.971, Validation Accuracy:  0.924, Loss:  0.022
Epoch   2 Batch  757/1077 - Train Accuracy:  0.964, Validation Accuracy:  0.927, Loss:  0.026
Epoch   2 Batch  758/1077 - Train Accuracy:  0.975, Validation Accuracy:  0.926, Loss:  0.017
Epoch   2 Batch  759/1077 - Train Accuracy:  0.961, Validation Accuracy:  0.931, Loss:  0.024
Epoch   2 Batch  760/1077 - Train Accuracy:  0.940, Validation Accuracy:  0.930, Loss:  0.029
Epoch   2 Batch  761/1077 - Train Accuracy:  0.960, Validation Accuracy:  0.944, Loss:  0.030
Epoch   2 Batch  762/1077 - Train Accuracy:  0.953, Validati

Epoch   2 Batch  840/1077 - Train Accuracy:  0.957, Validation Accuracy:  0.944, Loss:  0.022
Epoch   2 Batch  841/1077 - Train Accuracy:  0.946, Validation Accuracy:  0.944, Loss:  0.033
Epoch   2 Batch  842/1077 - Train Accuracy:  0.956, Validation Accuracy:  0.951, Loss:  0.028
Epoch   2 Batch  843/1077 - Train Accuracy:  0.962, Validation Accuracy:  0.952, Loss:  0.021
Epoch   2 Batch  844/1077 - Train Accuracy:  0.972, Validation Accuracy:  0.950, Loss:  0.023
Epoch   2 Batch  845/1077 - Train Accuracy:  0.940, Validation Accuracy:  0.950, Loss:  0.024
Epoch   2 Batch  846/1077 - Train Accuracy:  0.959, Validation Accuracy:  0.948, Loss:  0.036
Epoch   2 Batch  847/1077 - Train Accuracy:  0.955, Validation Accuracy:  0.942, Loss:  0.028
Epoch   2 Batch  848/1077 - Train Accuracy:  0.961, Validation Accuracy:  0.942, Loss:  0.024
Epoch   2 Batch  849/1077 - Train Accuracy:  0.936, Validation Accuracy:  0.944, Loss:  0.021
Epoch   2 Batch  850/1077 - Train Accuracy:  0.941, Validati

Epoch   2 Batch  928/1077 - Train Accuracy:  0.959, Validation Accuracy:  0.945, Loss:  0.026
Epoch   2 Batch  929/1077 - Train Accuracy:  0.951, Validation Accuracy:  0.945, Loss:  0.025
Epoch   2 Batch  930/1077 - Train Accuracy:  0.950, Validation Accuracy:  0.940, Loss:  0.024
Epoch   2 Batch  931/1077 - Train Accuracy:  0.957, Validation Accuracy:  0.942, Loss:  0.023
Epoch   2 Batch  932/1077 - Train Accuracy:  0.950, Validation Accuracy:  0.948, Loss:  0.024
Epoch   2 Batch  933/1077 - Train Accuracy:  0.947, Validation Accuracy:  0.947, Loss:  0.026
Epoch   2 Batch  934/1077 - Train Accuracy:  0.958, Validation Accuracy:  0.949, Loss:  0.022
Epoch   2 Batch  935/1077 - Train Accuracy:  0.969, Validation Accuracy:  0.936, Loss:  0.024
Epoch   2 Batch  936/1077 - Train Accuracy:  0.945, Validation Accuracy:  0.945, Loss:  0.028
Epoch   2 Batch  937/1077 - Train Accuracy:  0.972, Validation Accuracy:  0.938, Loss:  0.033
Epoch   2 Batch  938/1077 - Train Accuracy:  0.967, Validati

Epoch   2 Batch 1016/1077 - Train Accuracy:  0.947, Validation Accuracy:  0.917, Loss:  0.035
Epoch   2 Batch 1017/1077 - Train Accuracy:  0.947, Validation Accuracy:  0.928, Loss:  0.030
Epoch   2 Batch 1018/1077 - Train Accuracy:  0.933, Validation Accuracy:  0.936, Loss:  0.026
Epoch   2 Batch 1019/1077 - Train Accuracy:  0.930, Validation Accuracy:  0.945, Loss:  0.041
Epoch   2 Batch 1020/1077 - Train Accuracy:  0.969, Validation Accuracy:  0.942, Loss:  0.025
Epoch   2 Batch 1021/1077 - Train Accuracy:  0.951, Validation Accuracy:  0.938, Loss:  0.024
Epoch   2 Batch 1022/1077 - Train Accuracy:  0.955, Validation Accuracy:  0.933, Loss:  0.025
Epoch   2 Batch 1023/1077 - Train Accuracy:  0.944, Validation Accuracy:  0.936, Loss:  0.032
Epoch   2 Batch 1024/1077 - Train Accuracy:  0.939, Validation Accuracy:  0.941, Loss:  0.038
Epoch   2 Batch 1025/1077 - Train Accuracy:  0.931, Validation Accuracy:  0.922, Loss:  0.027
Epoch   2 Batch 1026/1077 - Train Accuracy:  0.961, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
- Convert words not in the vocabulary, to the `<UNK>` word id.

In [30]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sentence.lower()
    id_sentence = [vocab_to_int[word] if word in vocab_to_int else vocab_to_int['<UNK>'] for word in sentence.split()]
    return id_sentence


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [31]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('logits:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence], keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in np.argmax(translate_logits, 1)]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)]))

Input
  Word Ids:      [208, 150, 184, 25, 93, 90, 168]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [144, 244, 299, 49, 104, 172, 205, 157, 1]
  French Words: ['il', 'a', 'vu', 'un', 'vieux', 'camion', 'jaune', '.', '<EOS>']


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  Additionally, the translations in this data set were made by Google translate, so the translations themselves aren't particularly good.  (We apologize to the French speakers out there!) Thankfully, for this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.